# This is the code used to generate the boards in my paper "Build and Conquer: Solving N Queens Problem using Iterative Compression" -the new state-of-art in generating all solutions in conjecture- to read it open the link https://ieeexplore.ieee.org/document/9070976

In [2]:
from math import factorial
from PIL import Image, ImageDraw, ImageFont
from itertools import cycle
from copy import deepcopy

In [3]:
def nonattacked_corners(queens):
    n = len(queens) + 1 # n stands for number of corners = number of queens + 1
    nonattacked = []
    corners = []
    
    for i in range(n):
        corners.append([])
        for j in range(n):
            corners[i].append(0) # 0 for nonattacked, 1 for attacked
    
    for q in queens:
        a = q[0]
        b = q[1]
        x = q[0] + 1
        y = q[1] + 1
        
        while a>=0 or b>=0 or x<n or y<n :
            if a>=0:
                if b>=0:
                    corners[a][b] = 1
                if y<n :
                    corners[a][y] = 1
                    
            if x<n:
                if b>=0:
                    corners[x][b] = 1
                if y<n :
                    corners[x][y] = 1
                    
            a -= 1
            b -= 1
            x += 1
            y += 1
            
    for i in range(n):
        for j in range(n):
            if corners[i][j] == 0:
                nonattacked.append([i,j])
                
    return nonattacked

def add_queen(queens_old, queen):
    queens = deepcopy(queens_old)
    for q in queens:
        if q[0] >= queen[0]:
            q[0] += 1
        if q[1] >= queen[1]:
            q[1] += 1
    queens.append(queen)
    return sorted(queens)

def is_solution(queens):
    queens = sorted(queens)
    for i in range(len(queens)):
        if i != queens[i][0]:
            return False
        for j in range(i+1, len(queens)):
            if (queens[i][1] == queens[j][1]) or ((queens[j][0] - queens[i][0]) == abs(queens[i][1] - queens[j][1])):
                return False
    return True

def rotate_90(queens):
    n = len(queens)
    rotated_queens = deepcopy(queens)
    for q in rotated_queens:
        q[0], q[1] = q[1], n-1 - q[0]   
    return sorted(rotated_queens)

def mirror(queens):
    n = len(queens)
    mirrored_queens = deepcopy(queens)
    for q in mirrored_queens:
        q[0] = n-1 - q[0]   
    return sorted(mirrored_queens)

def get_similar_solutions(queens): # get all the rotated and mirrored versions of the solution
    solutions = [deepcopy(queens)]
    rotation = deepcopy(queens)
    
    for i in range(3):
        rotation = rotate_90(rotation)
        if rotation not in solutions:
            solutions.append(rotation)
            
    rotation = mirror(queens)
    if rotation not in solutions:
        solutions.append(rotation)        
    
    for i in range(3):
        rotation = rotate_90(rotation)
        if rotation not in solutions:
            solutions.append(rotation)
    
    return solutions

def draw_and_save_board(queens, pixel_width=500, name=None): 
    # shout out to victorfei
    # this function is a modified version of the chessboard drawing function written by victorfei
    # original function : https://gist.github.com/victorfei/1843ffd5fe871ef74d6bb3ce2a01dee8
    n = len(queens)
    nonattacked = nonattacked_corners(queens)
    
    def sq_start(i):
        return i*pixel_width / n

    def square(i, j):
        return map(sq_start, [i, j, i+1, j+1])

    image = Image.new("RGB", (pixel_width, pixel_width))
    font = ImageFont.truetype("arial.ttf", int(pixel_width/(1.5*n)))
    draw_square = ImageDraw.Draw(image).rectangle
    draw_queen = ImageDraw.Draw(image).text
    draw_circle = ImageDraw.Draw(image).ellipse
    squares = (square(i,j)
               for i_start, j in zip(cycle((0, 1)), range(n))
               for i in range(i_start, n, 2))
    
    for sq in squares:
        draw_square(list(sq), fill='#deb887')
        
    for q in queens:
        x_start = sq_start(q[1])
        y_start = sq_start(q[0])
        draw_queen((x_start + int(6*pixel_width/(25*n)), y_start + int(7*pixel_width/(50*n))), 'Q', fill='white', font=font)
    
    radius = int(pixel_width/(10*n))
    for i in range(n+1):
        for j in range(n+1):
            x = sq_start(i)
            y = sq_start(j)
            
            if [i,j] in nonattacked:
                if i == 0:
                    x_start = x
                    x_end = x + 2*radius
                elif i == n:
                    x_start = x - 2*radius
                    x_end = x
                else :
                    x_start = x - radius
                    x_end = x + radius
                    
                if j == 0:
                    y_start = y
                    y_end = y + 2*radius
                elif j == n:
                    y_start = y - 2*radius
                    y_end = y
                else :
                    y_start = y - radius
                    y_end = y + radius
                
                draw_square([x_start, y_start, x_end, y_end], fill='green')
            else :
                draw_circle([x-radius, y-radius, x+radius, y+radius], fill='gray')
    
    if name == None:
        image.save("chessboard.png")
    else :
        image.save('boards/'+name)

In [4]:
current_order = 3
boards = [[[[0,0], [1,2], [2,1]]]] # have all distinct solutions and psuedo-solutions
solutions = [] # have only distinct solutions
fin_order = 8

draw_and_save_board([[0,0], [1,2], [2,1]], name='root_board.png')

for i in range(fin_order-current_order):
    boards.append([])
    solutions.append([])
    for b in boards[i]:
        nonattacked = nonattacked_corners(b)
        for q in nonattacked:
            new_board = add_queen(b, q)
            instances = get_similar_solutions(new_board)
            is_added = False
            for ins in instances:
                if ins in boards[i+1]:
                    is_added = True
                    break
            if not is_added:
                if is_solution(new_board):
                    solutions[i].append(new_board)
                    boards[i+1].append(new_board)
                    draw_and_save_board(new_board, name="solution "+str(i+1+current_order)+"_"+str(len(solutions[i]))+'.png')
                else:
                    boards[i+1].append(new_board)
                    draw_and_save_board(new_board, name="board "+str(i+1+current_order)+"_"+str(len(boards[i+1]))+'.png')